In [ ]:
# Uncomment only when running first time

# !pip install spotipy
# !pip install bar_chart_race
# !pip install wordcloud

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly
import bar_chart_race as bcr
from wordcloud import WordCloud
import warnings

# Methods to Download and Format Data from Spotify API

In [ ]:
# Change the client id and secret (we will work on this during meeting)
# Steps:
# 1. Log into Spotify for Developers using Spotify Login
# 2. Go to Dashboard and Create New App (You can name it anything and put random info for the other fields)
# 3. Under App Name, there will be the Client ID and a "Show Secret" Button which will show the Client Secret When Clicked

client_id = "172109b5136c4b7fb9303dfb4135c198"
client_secret = "17b1fadaec064a99a2b5deb98c1f939e"

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# Gets Spotify Username and Playlist IDs 
# To get Playlist URI, click on "..." -> then on "Share" -> then "Copy link to playlist"
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

In [ ]:
# Gets Spotify Singular Song Data From These Locations
# https://developer.spotify.com/documentation/web-api/reference/#/operations/get-track
# https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)

    # Song Meta Data (Most Known Features)
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    # Song Features (Details)
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    valence = features[0]['valence']
    time_signature = features[0]['time_signature']
    key = features[0]['key']
    mode = features[0]['mode']
    uri = features[0]['uri']

    track = [name, album, artist, release_date,
             length, popularity, acousticness,
             danceability, energy, instrumentalness, 
             liveness, loudness, speechiness, tempo, 
             valence, time_signature,
             key, mode, uri]
    return track

In [ ]:
# Loops Over Track IDs to Get All Songs in Playlist
def loop_playlist(playlist_ids):
    tracks = []
    for i in range(len(playlist_ids)):
        time.sleep(.2)
        track = getTrackFeatures(playlist_ids[i])
        tracks.append(track)
    return tracks

In [ ]:
# Turns Data Into Dataframe
def get_spotify_df(tracks):
    df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date',
                                         'length', 'popularity', 'acousticness', 'danceability',
                                         'energy', 'instrumentalness', 'liveness', 'loudness',
                                         'speechiness', 'tempo', 'valence', 'time_signature',
                                         'key', 'mode', 'uri'])
    return df

In [ ]:
def get_years(df):
    years = []
    for date in df['release_date'].values:
        if '-' in date:
            years.append(date.split('-')[0])
        else:
            years.append(date)
    df['release_year'] = years
    return df

### Getting Song Data From Playlist

###### Playlist 1

In [ ]:
# Change Playlist URI and Username
workout_playlist = "https://open.spotify.com/playlist/0mi1TGgyw1DqcQMPcFGvgo?si=92cb790224b54150"
ids_workout = getTrackIDs("Shreya Krishnan", workout_playlist)
time_start = time.time()
ids_workout_loop = loop_playlist(ids_workout)
time_end = time.time()
# Prints out time taken to loop through playlist in minutes (as playlist gets longer, might take closer to 4-5 minutes)
print((time_end - time_start)/60)

In [ ]:
# Converts Data to Dataframe
df_workout = get_spotify_df(ids_workout_loop)
df_workout = get_years(df_workout)
df_workout

In [ ]:
# Saves DataFrame to csv File
df_workout.to_csv(r"C:\Users\ayerh\Documents\College Stuff\AIM Project\Intro Project\Workout_Playlist_Data.csv")

###### Playlist 2

In [ ]:
# Change Playlist URI and Username
mellow_playlist = "https://open.spotify.com/playlist/01jBys1BHC938RLYNkr7Vp?si=fa0775343e2c4ff9"
ids_mellow = getTrackIDs("Shreya Krishnan", mellow_playlist)
time_start = time.time()
ids_mellow_loop = loop_playlist(ids_mellow)
time_end = time.time()
# Prints out time taken to loop through playlist in minutes (as playlist gets longer, might take closer to 4-5 minutes)
print((time_end - time_start)/60)

In [ ]:
# Converts Data to Dataframe
df_mellow = get_spotify_df(ids_mellow_loop)
df_mellow = get_years(df_mellow)
df_mellow

###### Playlist 3

In [ ]:
# Change Playlist URI and Username
study_playlist = "https://open.spotify.com/playlist/4IwQfCoPvBLJXdZSKfdYZy?si=369f40e3b74d4fab"
ids_study = getTrackIDs("Shreya Krishnan", study_playlist)
time_start = time.time()
ids_study_loop = loop_playlist(ids_study)
time_end = time.time()
# Prints out time taken to loop through playlist in minutes (as playlist gets longer, might take closer to 4-5 minutes)
print((time_end - time_start)/60)

In [ ]:
# Converts Data to Dataframe
df_study = get_spotify_df(ids_study_loop)
df_study = get_years(df_study)
df_study

# Data Visualization

###### Playlist 1

###### Playlist 2

###### Playlist 3

### Comparing Artists in Playlists